In [32]:
!pip install snowflake-connector-python

In [33]:
!pip install snowflake-snowpark-python

In [34]:
!pip install pandas

In [35]:
!pip install pyarrow

In [36]:
!pip install matplotlib

In [37]:
!pip install seaborn

In [38]:
!pip install numpy

In [39]:
!pip install scipy

In [40]:
!pip install scikit-learn

In [41]:
!pip install statsmodels

In [42]:
!pip install tensorflow

In [43]:
!pip install keras

In [44]:
!pip install xgboost

In [45]:
!pip install lightgbm

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/21/1b/550ee378512b78847930f5d74228ca1fdba2a7fbdeaac9aeccc085b0e257/lightgbm-4.6.0-py3-none-macosx_12_0_arm64.whl.metadata
  Using cached lightgbm-4.6.0-py3-none-macosx_12_0_arm64.whl.metadata (17 kB)
Using cached lightgbm-4.6.0-py3-none-macosx_12_0_arm64.whl (1.6 MB)


In [46]:
!pip install catboost

In [47]:
!pip install mlflow

In [48]:
!pip install pydot

In [49]:
!pip install 'snowflake-snowpark-python[ml]'

### IMPORTS

In [2]:
from snowflake.snowpark.session import Session
import getpass
import pandas as pd
import json
import os

### CREATE SNOWPARK SESSION

In [8]:
# WGDNNWJ-OD50189
accountname = getpass.getpass() # ORGNAME-ACCOUNTNAME (separated by minus sign)
print(accountname)

WGDNNWJ-OD50189


In [9]:
username = getpass.getpass()    # SNOWFLAKE-USERNAME
password = getpass.getpass()    # SNOWFLAKE-PASSWORD

In [10]:
connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "ACCOUNTADMIN"
}

session = Session.builder.configs(connection_parameters).create()

### Create Virtual Warehouse (Compute) 

In [ ]:
# Create two Virtual Warehouses
session.sql("CREATE OR REPLACE WAREHOUSE CC_WH_HIGH_MEM WITH WAREHOUSE_SIZE='MEDIUM' WAREHOUSE_TYPE='SNOWPARK-OPTIMIZED'").collect()
session.sql("CREATE OR REPLACE WAREHOUSE CC_WH WITH WAREHOUSE_SIZE='X-SMALL'").collect()

[Row(status='Warehouse CC_WH successfully created.')]

### Create Database in Snowflake

In [7]:
# Create Databse
session.sql("CREATE OR REPLACE DATABASE CC_DB").collect()

[Row(status='Database CC_DB successfully created.')]

### Load Kaggle Dataset into Snowflake DB

In [8]:
# Loading from local CSV-files
application_record_df = pd.read_csv('RawData/application_record.csv.zip')
credit_record_df = pd.read_csv('RawData/credit_record.csv.zip')

In [9]:
# Upload to Snowflake
session.write_pandas(application_record_df, table_name='APPLICATION_RECORD', auto_create_table=True, overwrite=True)
session.write_pandas(credit_record_df, table_name='CREDIT_RECORD', auto_create_table=True, overwrite=True)

In [10]:
connection_parameters["database"] = 'CC_DB'
connection_parameters["schema"] = 'PUBLIC'
connection_parameters["warehouse"] = 'CC_WH'
with open('creds.json', 'w') as outfile:
        outfile.write(json.dumps(connection_parameters))

In [11]:
session.close()